In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("processed data.csv")
data = data.drop(columns = ['Unnamed: 0'])

In [3]:
# Comp = name of league
data['Comp'].replace(data['Comp'].unique(),[1, 3, 2, 4, 5], inplace=True)
# drop name column and preferred_foot
data = data.drop(columns = ['name','preferred_foot'])

In [4]:
#convert the categorical variables to dummies
data = pd.get_dummies(data) 
# log tranformation for market vakue
data['market value'] = np.log(data['market value'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Volleys,age,overall,potential,market value,wage value,position common_defender,position common_goalkeeper,position common_midfielder,position common_striker
0,34,2983,0.06,0.00,0.15,0.03,1,0.54,37,81,...,24,33,76,76,15.096444,23000.0,1,0,0,0
1,31,2462,0.04,0.00,0.44,0.11,1,0.66,50,79,...,38,21,70,75,14.603968,7000.0,0,0,1,0
2,34,2956,0.00,0.06,0.27,0.00,1,0.91,77,76,...,37,28,75,75,15.404746,18000.0,0,0,1,0
3,24,726,0.00,0.12,0.37,0.00,3,2.22,81,39,...,53,22,68,76,14.808762,9000.0,0,0,0,1
4,30,2536,0.14,0.00,0.07,0.04,2,0.57,64,77,...,39,33,83,83,16.677711,75000.0,1,0,0,0


In [5]:
data_build = data.copy()
y = data_build['market value']
x = data_build.drop(columns =['market value'])

In [6]:
# Standardization
from sklearn.preprocessing import StandardScaler

def standard_scale(x):
    convert = StandardScaler().fit(x)
    return convert.transform(x)
x = standard_scale(x)
x

array([[ 1.2134208 ,  1.66583221, -0.25109419, ..., -0.30265996,
        -0.79642056, -0.4737772 ],
       [ 0.93994692,  1.12245649, -0.34362811, ..., -0.30265996,
         1.25561801, -0.4737772 ],
       [ 1.2134208 ,  1.63767262, -0.52869597, ..., -0.30265996,
         1.25561801, -0.4737772 ],
       ...,
       [-1.79479184, -1.41503508, -0.52869597, ...,  3.30403793,
        -0.79642056, -0.4737772 ],
       [ 1.39573672,  1.25699674, -0.20482722, ..., -0.30265996,
         1.25561801, -0.4737772 ],
       [ 1.39573672,  1.45932858,  0.5354442 , ..., -0.30265996,
         1.25561801, -0.4737772 ]])

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size = 0.33, random_state = 3 )

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
regressor = SVR()
params = {'kernel': ['linear', 'rbf', 'sigmoid'],
          'C' : [0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10, 20, 30, 50, 100, 200, 500],
         'epsilon': [0, 0.1, 0.2, 0.3] }
scoring_fnc = 'neg_mean_squared_error'
grid = GridSearchCV(estimator=regressor, param_grid=params, scoring= scoring_fnc, cv= 5 , verbose= 1)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 204 candidates, totalling 1020 fits


In [ ]:
# get best parameter
grid.best_estimator_.get_params()

In [ ]:
#  Training model with hyperparameter tuning
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
model = SVR( C =5.0, kernel = 'rbf',epsilon = 0 )
model.fit(x_train, y_train)
y_pre = model.predict(x_test)
print('Root mean squared' ,(mean_squared_error(y_pre,y_test )**(1/2)))
print('R2-scored ', r2_score(y_pre, y_test))